In [1]:
# %load retinanet_train.py
#!/usr/bin/env python

# In[1]:

print("Start importing package...")
import time
import os
import copy
import argparse
import pdb
import collections
import sys

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms
import torchvision

import model
import mymodel
from anchors import Anchors
import losses
from dataloader import CocoDataset, CSVDataset, collater, Resizer, AspectRatioBasedSampler, Augmenter, UnNormalizer, Normalizer
from torch.utils.data import Dataset, DataLoader
from mydataloader import MyDataset
import coco_eval
import csv_eval
import visdom
vis = visdom.Visdom(env='retinanet_seresnextTest')

assert torch.__version__.split('.')[1] == '4'

print('CUDA available: {}'.format(torch.cuda.is_available()))
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
#参数
NAME="RSNA"
DATA_PATH = "/data/krf/dataset"
CSV_TRAINS = [DATA_PATH + "/test_csv_train0.csv",DATA_PATH + "/test_csv_train1.csv",DATA_PATH + "/test_csv_train2.csv",DATA_PATH + "/test_csv_train3.csv"]
CSV_VALS = [DATA_PATH + "/test_csv_val0.csv",DATA_PATH + "/test_csv_val1.csv",DATA_PATH + "/test_csv_val2.csv",DATA_PATH + "/test_csv_val3.csv"]
CSV_CLASSES = DATA_PATH + "/test_classes.csv"
DEPTH = 50
EPOCHS = 20
BATCH_SIZE=2
VAL_SIZE = 30
#TRAIN_SIZE = 100
#数据预处理
import csv
import random
from tqdm import tqdm

Start importing package...
CUDA available: True


In [2]:
# with open(DATA_PATH+"/stage_1_train_labels.csv") as f:
#     reader = csv.reader(f)
#     rows=[row for row in  reader]
#     rows = rows[1:]
#     random.shuffle(rows)
#     for row in rows:
#         row[0] = DATA_PATH+"/stage_1_train_images/"+row[0]+".dcm"
#         if row[1] == '' and row[2] == '' and row[3] == '' and row[4] == '':
#             row[5] = ''
#         else:
#             row[3] = str(float(row[1]) + float(row[3]))# x2 = x1 + w 
#             row[4] = str(float(row[2]) + float(row[4]))# y2 = y1 + h
#     val_rows = rows[:VAL_SIZE]
#     train_rows = rows[VAL_SIZE:]
#     print(len(val_rows),len(train_rows))
#     with open(CSV_TRAIN,'w') as f2:
#         write = csv.writer(f2)
#         write.writerows(train_rows)
#         print("csv_train 写入完毕")
#     with open(CSV_VAL,'w') as f3:
#         write = csv.writer(f3)
#         write.writerows(val_rows)
#         print("csv_val 写入完毕")

# with open(CSV_CLASSES,'w') as f:
#     write = csv.writer(f)
    
#     row = ['0','0']
#     write.writerow(row)
#     row = ['1','1']
#     write.writerow(row)
#     row = ['2','2']
#     write.writerow(row)
#     print("csv_classes 写入完毕")



# # #每次跑这个函数之前需要先删除之前的

# df = pd.read_csv(DATA_PATH+"/stage_1_train_labels.csv")
# ddf = pd.read_csv(DATA_PATH+'/stage_1_detailed_class_info.csv')
# train_images = os.listdir(DATA_PATH+"/stage_1_train_images")[:500]

# random.shuffle(train_images)#打乱图片顺序
# count = 0
# pos_cnt_train = [0,0,0,0]
# pos_cnt_val = [0,0,0,0]
# class_dict = {'Normal':0,'Lung Opacity':1,'No Lung Opacity / Not Normal':2}
# for img_name in tqdm(train_images):
#     results = df[df['patientId']==img_name.split('.')[0]].values
#     detail = ddf[df['patientId']==img_name.split('.')[0]].values
#     for row in results:
#         row[0] = DATA_PATH+"/stage_1_train_images/"+row[0]+".dcm"
#         if row[5] == 1:
#             pos_cnt_val[count % 4] += 1
#         if row[1] >= 0 and row[1] <= 1024:
#             row[3] = str(float(row[1]) + float(row[3]))# x2 = x1 + w 
#             row[4] = str(float(row[2]) + float(row[4]))# y2 = y1 + h
#         else:
#             row[1] = ''
#             row[2] = ''
#             row[3] = ''
#             row[4] = ''
#             row[5] = class_dict[detail[0,1]]
#         with open(CSV_VALS[count % 4],'a') as f:
#             write = csv.writer(f)
#             write.writerow(row)
#         for i in range(4):
#             if count % 4 == i:
#                 continue
#             else:
#                 with open(CSV_TRAINS[count % 4],'a') as f:
#                     write = csv.writer(f)
#                     write.writerow(row) 
#     count += 1
# print(pos_cnt_train,pos_cnt_val)  

In [3]:

print("Loading data...")

#%time
#制作数据loader
dataset_train = []
dataset_val = []

for i in range(4): 
    dataset_train.append(MyDataset(train_file=CSV_TRAINS[i], class_list=CSV_CLASSES, transform=transforms.Compose([Normalizer(), Augmenter(), Resizer()])))
    dataset_val.append(MyDataset(train_file=CSV_VALS[i], class_list=CSV_CLASSES, transform=transforms.Compose([Normalizer(), Resizer()])))

#每次的sampler的参数：来源、batchsize、是否抛弃最后一层？？？

# num_workers 同时工作的组？collater:校验用的吧
dataloader_train = []
dataloader_val = []

for i in range(4):
    sampler = AspectRatioBasedSampler(dataset_train[i], batch_size=BATCH_SIZE, drop_last=False)
    sampler_val = AspectRatioBasedSampler(dataset_val[i], batch_size=1, drop_last=False)
    dataloader_train.append(DataLoader(dataset_train[i], num_workers=1, collate_fn=collater, batch_sampler=sampler))
    dataloader_val.append(DataLoader(dataset_val[i], num_workers=1, collate_fn=collater, batch_sampler=sampler_val))



Loading data...


In [ ]:
retinanets = []
# Create the model
# if DEPTH == 18:
#     retinanet = model.resnet18(num_classes=dataset_train.num_classes(), pretrained=True)
# elif DEPTH == 34:
#     retinanet = model.resnet34(num_classes=dataset_train.num_classes(), pretrained=True)
# elif DEPTH == 50:
#     for i in range(4):
#         retinanets.append(model.resnet50(num_classes=dataset_train[i].num_classes(), pretrained=True))
# elif DEPTH == 101:
#     for i in range(4):
#         retinanets.append(model.resnet101(num_classes=dataset_train[i].num_classes(), pretrained=True)) 
# elif DEPTH == 152:
#     for i in range(4):
#         retinanets.append(model.resnet152(num_classes=dataset_train[i].num_classes(), pretrained=True))
# else:
#     raise ValueError('Unsupported model depth, must be one of 18, 34, 50, 101, 152')
#retinanet = torch.load('weights/RSNA_retinanet_5.pt')
for i in range(2):
    retinanets.append(mymodel.se_resnext50_32x4d(num_classes=dataset_train[i].num_classes()))
optimizer = []
scheduler = []
loss_hist = []
classloss_hist = []
regressloss_hist = []
wholeclassloss_hist = []
for i in range(2):
    retinanets[i] = retinanets[i].cuda()
    #变成并行
    retinanets[i] = torch.nn.DataParallel(retinanets[i]).cuda()
    #训练模式
    retinanets[i].training = True
    #学习率0.00001  
    optimizer.append(optim.Adam(retinanets[i].parameters(), lr=1e-4))
    #如果3个epoch损失没有减少则降低学习率
    scheduler.append(optim.lr_scheduler.ReduceLROnPlateau(optimizer[i], patience=2, verbose=True))
    # TODO 这是干什么 deque是为了高效实现插入和删除操作的双向列表，适合用于队列和栈：这里是定义了一个500的队列
    loss_hist.append(collections.deque(maxlen=500))
    classloss_hist.append(collections.deque(maxlen=500))
    regressloss_hist.append(collections.deque(maxlen=500))
    wholeclassloss_hist.append(collections.deque(maxlen=500))
    print('Num training images: {}'.format(len(dataset_train[i])))
# In[5]:



Num training images: 125
Num training images: 125


In [ ]:
VAL_SIZE = 15
count = [0,0]
import traceback
for epoch_num in range(EPOCHS):

    for i in range(2):
        retinanets[i].train()
        retinanets[i].module.freeze_bn()
        epoch_loss = []
        
        for iter_num, data in enumerate(dataloader_train[i]):
            if 1 == 1:
            #try:
                optimizer[i].zero_grad()

                focal_loss, whole_class_loss = retinanets[i]([Variable(data['img'].cuda().float()), Variable(data['annot'].cuda())])
                classification_loss, regression_loss = focal_loss
                classification_loss = classification_loss.mean()
                regression_loss = regression_loss.mean()
                whole_class_loss = whole_class_loss.mean()
                loss = classification_loss + regression_loss + whole_class_loss

                if bool(loss == 0):
                    continue
                #反向传播？
                loss.backward()

                #这是干嘛？？TODO
                torch.nn.utils.clip_grad_norm_(retinanets[i].parameters(), 0.1)

                #这?TODO
                optimizer[i].step()

                loss_hist[i].append(float(loss))
                classloss_hist[i].append(float(classification_loss))
                regressloss_hist[i].append(float(regression_loss))
                wholeclassloss_hist[i].append(float(whole_class_loss))
                epoch_loss.append(float(loss))
                
                print('Epoch: {} | Iteration: {} | Classification loss: {:1.5f} | Regression loss: {:1.5f} | Whole Class loss: {:1.5f} | Running loss: {:1.5f}'.format(epoch_num, iter_num, np.mean(classloss_hist[i]), np.mean(regressloss_hist[i]), np.mean(wholeclassloss_hist[i]), np.mean(loss_hist[i])))
                vis.line(X=torch.Tensor([count[i]]), Y=torch.Tensor([np.mean(loss_hist[i])]), win='train loss '+str(i), update='append' ,opts={'title':'train loss '+str(i)})
                vis.line(X=torch.Tensor([count[i]]), Y=torch.Tensor([np.mean(classloss_hist[i])]), win='classification loss '+str(i), update='append' ,opts={'title':'classification loss '+str(i)})
                vis.line(X=torch.Tensor([count[i]]), Y=torch.Tensor([np.mean(regressloss_hist[i])]), win='regression loss '+str(i), update='append' ,opts={'title':'regression loss '+str(i)})
                vis.line(X=torch.Tensor([count[i]]), Y=torch.Tensor([np.mean(wholeclassloss_hist[i])]), win='whole class loss '+str(i), update='append' ,opts={'title':'whole class loss '+str(i)})

                count[i] += 1
                vis.save(['retinanet_seresnextTest'])

                del classification_loss
                del regression_loss
#                 print(count[i])
#                 if count[i] % VAL_SIZE == 0:
#                     print("Evaluating dataset")
#                     retinanets[i].eval()
#                     mAP = csv_eval.evaluate(dataset_val[i],retinanets[i])
#                     vis.line(X=torch.Tensor([count[i]]), Y=torch.Tensor([mAP[0][0]]), win='val mAP '+str(i), update='append' ,opts={'title':'mAP val '+str(i)})
#                     vis.save(['retinanet_seresnextTest'])
#                     #torch.save(retinanets[i].module, 'weights_stage1/{}_seresnext101_{}.pt'.format(i, epoch_num))
#                     retinanets[i].train()
#             except Exception as e:
#                 print(e)
#                 continue


        print("Evaluating dataset")
        retinanets[i].eval()
    #     for index,data in enumerate(dataloader_val):
    #         #data = dataset[index]
    #         scale = data['scale']
    #         # run network
    #         #scores, labels, boxes = retinanet(data['img'].permute(2, 0, 1).cuda().float().unsqueeze(dim=0))
    #         print(Variable(data['img'].cuda()))
    #         scores, labels, boxes = retinanet(Variable(data['img'].cuda()))
    #         #mAP = csv_eval.evaluate(dataset_val,retinanet)
#         try:
        if 1 == 1:
            mAP = csv_eval.evaluate(dataset_val[i],retinanets[i])
            vis.line(X=torch.Tensor([epoch_num]), Y=torch.Tensor([mAP[0][0]]), win='val mAP '+str(i), update='append' ,opts={'title':'mAP val '+str(i)})
            vis.save(['retinanet_seresnextTest'])
#         except Exception as e:
#             print(e)
#             print( 'traceback.print_exc():', traceback.print_exc())
#             continue
        #这一步也看不懂？？TODO 
        scheduler[i].step(np.mean(epoch_loss))

        #torch.save(retinanets[i].module, 'weights_stage1/{}_seresnext101_{}.pt'.format(i, epoch_num))
    #     torch.save(retinanet.module, '{}_retinanet_{}.pt'.format(parser.dataset, epoch_num))

for i in range(4):
    retinanets[i].eval()
    #torch.save(retinanets[i], 'weights_stage1/{}_model_final_seresnext101.pt'.format(i))




/data/krf/anaconda/anaconda3/lib/python3.5/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")
/data/krf/anaconda/anaconda3/lib/python3.5/site-packages/torch/nn/parallel/_functions.py:58: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch: 0 | Iteration: 0 | Classification loss: 0.12567 | Regression loss: 0.00000 | Whole Class loss: 1.10193 | Running loss: 1.22761
Epoch: 0 | Iteration: 1 | Classification loss: 0.11312 | Regression loss: 0.00000 | Whole Class loss: 0.93985 | Running loss: 1.05297
Epoch: 0 | Iteration: 2 | Classification loss: 0.27994 | Regression loss: 0.18427 | Whole Class loss: 1.57832 | Running loss: 2.04253
Epoch: 0 | Iteration: 3 | Classification loss: 0.35777 | Regression loss: 0.26337 | Whole Class loss: 1.66089 | Running loss: 2.28203
Epoch: 0 | Iteration: 4 | Classification loss: 0.40823 | Regression loss: 0.32184 | Whole Class loss: 1.74484 | Running loss: 2.47491
Epoch: 0 | Iteration: 5 | Classification loss: 0.53369 | Regression loss: 0.44512 | Whole Class loss: 1.99557 | Running loss: 2.97437
Epoch: 0 | Iteration: 6 | Classification loss: 0.45953 | Regression loss: 0.38153 | Whole Class loss: 1.83435 | Running loss: 2.67541
Epoch: 0 | Iteration: 7 | Classification loss: 0.47503 | Regre

Epoch: 0 | Iteration: 61 | Classification loss: 0.28821 | Regression loss: 0.24527 | Whole Class loss: 1.23235 | Running loss: 1.76583
Epoch: 0 | Iteration: 62 | Classification loss: 0.28370 | Regression loss: 0.24138 | Whole Class loss: 1.22448 | Running loss: 1.74956
Evaluating dataset
125/125
mAP:
0: 0.0
1: 0.0
2: 0.0
Epoch: 0 | Iteration: 0 | Classification loss: 0.12567 | Regression loss: 0.00000 | Whole Class loss: 1.10191 | Running loss: 1.22758
Epoch: 0 | Iteration: 1 | Classification loss: 0.11656 | Regression loss: 0.00000 | Whole Class loss: 1.13244 | Running loss: 1.24899
Epoch: 0 | Iteration: 2 | Classification loss: 0.10614 | Regression loss: 0.00000 | Whole Class loss: 1.26377 | Running loss: 1.36991
Epoch: 0 | Iteration: 3 | Classification loss: 0.09712 | Regression loss: 0.00000 | Whole Class loss: 1.19012 | Running loss: 1.28724
Epoch: 0 | Iteration: 4 | Classification loss: 0.08563 | Regression loss: 0.00000 | Whole Class loss: 1.11171 | Running loss: 1.19734
Epoch: 

Epoch: 0 | Iteration: 59 | Classification loss: 0.26130 | Regression loss: 0.17514 | Whole Class loss: 1.27488 | Running loss: 1.71132
Epoch: 0 | Iteration: 60 | Classification loss: 0.25710 | Regression loss: 0.17227 | Whole Class loss: 1.27032 | Running loss: 1.69968
Epoch: 0 | Iteration: 61 | Classification loss: 0.25305 | Regression loss: 0.16949 | Whole Class loss: 1.26612 | Running loss: 1.68866
Epoch: 0 | Iteration: 62 | Classification loss: 0.25826 | Regression loss: 0.17482 | Whole Class loss: 1.26444 | Running loss: 1.69752
Evaluating dataset
125/125
mAP:
0: 0.0
1: 0.0
2: 0.0
Epoch: 1 | Iteration: 0 | Classification loss: 0.27931 | Regression loss: 0.23761 | Whole Class loss: 1.21592 | Running loss: 1.73283
Epoch: 1 | Iteration: 1 | Classification loss: 0.29252 | Regression loss: 0.24941 | Whole Class loss: 1.24494 | Running loss: 1.78687
Epoch: 1 | Iteration: 2 | Classification loss: 0.28814 | Regression loss: 0.24563 | Whole Class loss: 1.23654 | Running loss: 1.77031
Epoch

Epoch: 1 | Iteration: 57 | Classification loss: 0.27779 | Regression loss: 0.23467 | Whole Class loss: 1.20427 | Running loss: 1.71673
Epoch: 1 | Iteration: 58 | Classification loss: 0.27555 | Regression loss: 0.23275 | Whole Class loss: 1.20304 | Running loss: 1.71135
Epoch: 1 | Iteration: 59 | Classification loss: 0.27775 | Regression loss: 0.23439 | Whole Class loss: 1.20242 | Running loss: 1.71457
Epoch: 1 | Iteration: 60 | Classification loss: 0.27555 | Regression loss: 0.23250 | Whole Class loss: 1.19885 | Running loss: 1.70690
Epoch: 1 | Iteration: 61 | Classification loss: 0.27338 | Regression loss: 0.23064 | Whole Class loss: 1.19600 | Running loss: 1.70003
Epoch: 1 | Iteration: 62 | Classification loss: 0.27597 | Regression loss: 0.23274 | Whole Class loss: 1.19871 | Running loss: 1.70742
Evaluating dataset
125/125
mAP:
0: 0.0
1: 0.0
2: 0.0
Epoch: 1 | Iteration: 0 | Classification loss: 0.26365 | Regression loss: 0.17902 | Whole Class loss: 1.26420 | Running loss: 1.70687
Epo

...

Evaluating dataset
125/125
mAP:
0: 0.0
1: 0.04483916056382392
2: 0.0
Epoch: 13 | Iteration: 0 | Classification loss: 0.19055 | Regression loss: 0.15522 | Whole Class loss: 0.98454 | Running loss: 1.33031
Epoch: 13 | Iteration: 1 | Classification loss: 0.19146 | Regression loss: 0.15604 | Whole Class loss: 0.98546 | Running loss: 1.33296
Epoch: 13 | Iteration: 2 | Classification loss: 0.19235 | Regression loss: 0.15681 | Whole Class loss: 0.98724 | Running loss: 1.33640
Epoch: 13 | Iteration: 3 | Classification loss: 0.19128 | Regression loss: 0.15602 | Whole Class loss: 0.98694 | Running loss: 1.33424
Epoch: 13 | Iteration: 4 | Classification loss: 0.18995 | Regression loss: 0.15516 | Whole Class loss: 0.98370 | Running loss: 1.32881
Epoch: 13 | Iteration: 5 | Classification loss: 0.18995 | Regression loss: 0.15516 | Whole Class loss: 0.98385 | Running loss: 1.32896
Epoch: 13 | Iteration: 6 | Classification loss: 0.18879 | Regression loss: 0.15424 | Whole Class loss: 0.98122 | Running 

Epoch: 13 | Iteration: 60 | Classification loss: 0.18706 | Regression loss: 0.15260 | Whole Class loss: 0.96574 | Running loss: 1.30539
Epoch: 13 | Iteration: 61 | Classification loss: 0.18710 | Regression loss: 0.15260 | Whole Class loss: 0.96491 | Running loss: 1.30461
Epoch: 13 | Iteration: 62 | Classification loss: 0.18708 | Regression loss: 0.15260 | Whole Class loss: 0.96406 | Running loss: 1.30374
Evaluating dataset
125/125
mAP:
0: 0.0
1: 0.030294042750790156
2: 0.0
Epoch: 14 | Iteration: 0 | Classification loss: 0.23500 | Regression loss: 0.20585 | Whole Class loss: 0.96622 | Running loss: 1.40708
Epoch: 14 | Iteration: 1 | Classification loss: 0.23577 | Regression loss: 0.20673 | Whole Class loss: 0.96629 | Running loss: 1.40880
Epoch: 14 | Iteration: 2 | Classification loss: 0.23487 | Regression loss: 0.20583 | Whole Class loss: 0.96547 | Running loss: 1.40617
Epoch: 14 | Iteration: 3 | Classification loss: 0.23484 | Regression loss: 0.20574 | Whole Class loss: 0.96478 | Runn

Epoch: 14 | Iteration: 57 | Classification loss: 0.23941 | Regression loss: 0.20502 | Whole Class loss: 0.96561 | Running loss: 1.41003
Epoch: 14 | Iteration: 58 | Classification loss: 0.23832 | Regression loss: 0.20401 | Whole Class loss: 0.96431 | Running loss: 1.40663
Epoch: 14 | Iteration: 59 | Classification loss: 0.23832 | Regression loss: 0.20401 | Whole Class loss: 0.96399 | Running loss: 1.40632
Epoch: 14 | Iteration: 60 | Classification loss: 0.23832 | Regression loss: 0.20401 | Whole Class loss: 0.96033 | Running loss: 1.40266
Epoch: 14 | Iteration: 61 | Classification loss: 0.23850 | Regression loss: 0.20417 | Whole Class loss: 0.95729 | Running loss: 1.39996
Epoch: 14 | Iteration: 62 | Classification loss: 0.23850 | Regression loss: 0.20417 | Whole Class loss: 0.95778 | Running loss: 1.40046
Evaluating dataset
125/125
mAP:
0: 0.0
1: 0.0
2: 0.0
Epoch: 14 | Iteration: 0 | Classification loss: 0.18707 | Regression loss: 0.15260 | Whole Class loss: 0.96917 | Running loss: 1.30

Epoch: 14 | Iteration: 54 | Classification loss: 0.18164 | Regression loss: 0.14710 | Whole Class loss: 0.95022 | Running loss: 1.27895
Epoch: 14 | Iteration: 55 | Classification loss: 0.18163 | Regression loss: 0.14710 | Whole Class loss: 0.94976 | Running loss: 1.27849
Epoch: 14 | Iteration: 56 | Classification loss: 0.18250 | Regression loss: 0.14797 | Whole Class loss: 0.94791 | Running loss: 1.27837
Epoch: 14 | Iteration: 57 | Classification loss: 0.18248 | Regression loss: 0.14797 | Whole Class loss: 0.94736 | Running loss: 1.27781
Epoch: 14 | Iteration: 58 | Classification loss: 0.18371 | Regression loss: 0.14907 | Whole Class loss: 0.95525 | Running loss: 1.28802
Epoch: 14 | Iteration: 59 | Classification loss: 0.18371 | Regression loss: 0.14907 | Whole Class loss: 0.95468 | Running loss: 1.28746
Epoch: 14 | Iteration: 60 | Classification loss: 0.18294 | Regression loss: 0.14806 | Whole Class loss: 0.95287 | Running loss: 1.28387
Epoch: 14 | Iteration: 61 | Classification loss:

Epoch: 15 | Iteration: 51 | Classification loss: 0.23807 | Regression loss: 0.20234 | Whole Class loss: 0.93797 | Running loss: 1.37839
Epoch: 15 | Iteration: 52 | Classification loss: 0.23805 | Regression loss: 0.20234 | Whole Class loss: 0.94138 | Running loss: 1.38178
Epoch: 15 | Iteration: 53 | Classification loss: 0.23805 | Regression loss: 0.20234 | Whole Class loss: 0.94360 | Running loss: 1.38398
Epoch: 15 | Iteration: 54 | Classification loss: 0.23804 | Regression loss: 0.20234 | Whole Class loss: 0.94552 | Running loss: 1.38591
Epoch: 15 | Iteration: 55 | Classification loss: 0.23686 | Regression loss: 0.20130 | Whole Class loss: 0.94369 | Running loss: 1.38186
Epoch: 15 | Iteration: 56 | Classification loss: 0.23790 | Regression loss: 0.20213 | Whole Class loss: 0.94454 | Running loss: 1.38457
Epoch: 15 | Iteration: 57 | Classification loss: 0.23793 | Regression loss: 0.20213 | Whole Class loss: 0.94515 | Running loss: 1.38522
Epoch: 15 | Iteration: 58 | Classification loss: